In [4]:
# Import library
import sqlite3
import pandas as pd

# Create connection with SQLite database
conn = sqlite3.connect(":memory:")

# Create dataframes by importing reviews.csv and pricing_plans.csv, and transform them into sqlite file.
df = pd.read_csv('../input/reviews.csv')
df.to_sql('reviews', conn, if_exists='append', index=False)
print('ok')
df = pd.read_csv('../input/pricing_plans.csv')
df.to_sql('pricing_plans', conn, if_exists='append', index=False)
print('ok')

ok
ok


In [7]:
# Check the head of two dataframes
pd.read_sql("""

SELECT *
FROM reviews
LIMIT 50

""",con=conn)

pd.read_sql("""

SELECT *
FROM pricing_plans
LIMIT 50

""",con=conn)

,id,app_url,title,price,hint
0,0,https://apps.shopify.com/hide-paypal-and-other...,All in one,$0.99/month,None
1,1,https://apps.shopify.com/quick-pick,Quick Pick,$4.99/month,None
2,2,https://apps.shopify.com/printout-designer,Micro,$4.99/month,None
3,3,https://apps.shopify.com/printout-designer,Starter,$9.99/month,None
4,4,https://apps.shopify.com/printout-designer,Basic,$19.99/month,None
5,5,https://apps.shopify.com/printout-designer,Professional,$59.99/month,None
6,6,https://apps.shopify.com/cartsguru,Growth,$199/month,2.5% on sales recovered
7,7,https://apps.shopify.com/related-blog-posts,None,Free,None
8,8,https://apps.shopify.com/flits,Basic,Free to install,*Paid Add ons Available- Store credits & 3 others
9,9,https://apps.shopify.com/flits,Advance,$5/month,"you only pay what you’ve actually used, and no..."


In [8]:
# Check the count of different rating levels
pd.read_sql("""

SELECT rating,count(*) as [count]
FROM reviews
GROUP BY rating

""",con = conn)

,rating,count
0,1,8913
1,2,2552
2,3,5783
3,4,23507
4,5,246712


In [ ]:
# Check the count of different rating levels
pd.read_sql("""

SELECT rating,count(*) as [count]
FROM reviews
GROUP BY rating

""",con = conn)

In [9]:
# Join pricing,ratings and helpful_count by app_url.
pd.read_sql("""

    SELECT rating, helpful_count, price
    FROM reviews
    JOIN pricing_plans
    ON reviews.app_url = pricing_plans.app_url

""",con = conn)

,rating,helpful_count,price
0,1,0,$4.99/month
1,5,0,$4.99/month
2,5,0,$4.99/month
3,5,0,$4.99/month
4,5,0,$4.99/month
5,5,1,$29/month
6,5,1,$49/month
7,5,1,$79/month
8,5,1,$99/month
9,5,1,$29/month


In [11]:
# Check the ratings of apps with free charging.
pd.read_sql("""

SELECT *
FROM (
    SELECT rating, helpful_count, price
    FROM reviews
    JOIN pricing_plans
    ON reviews.app_url = pricing_plans.app_url
    )
WHERE price = 'Free' OR price = 'Free to install'
ORDER BY rating DESC

""",con=conn)

# We also want to know the count of different rating levels among free apps.
pd.read_sql("""

SELECT rating,count(*) as [count]
FROM (
    SELECT rating, helpful_count, price
    FROM reviews
    JOIN pricing_plans
    ON reviews.app_url = pricing_plans.app_url
    )
WHERE price = 'Free' OR price = 'Free to install'
GROUP BY rating
ORDER BY rating DESC

""",con=conn)
# We can see that most free apps are rated 5.

,rating,count
0,5,173433
1,4,18581
2,3,4531
3,2,1729
4,1,5907


In [12]:
# So what about ratings of charged apps?
pd.read_sql("""

SELECT *
FROM (
    SELECT rating, helpful_count, price
    FROM reviews
    JOIN pricing_plans
    ON reviews.app_url = pricing_plans.app_url
    )
WHERE price != 'Free' AND price != 'Free to install'
ORDER BY rating DESC

""",con=conn)

# and the count of their rating levels
pd.read_sql("""

SELECT rating,count(*) as [count]
FROM (
    SELECT rating, helpful_count, price
    FROM reviews
    JOIN pricing_plans
    ON reviews.app_url = pricing_plans.app_url
    )
WHERE price != 'Free' AND price != 'Free to install'
GROUP BY rating
ORDER BY rating DESC

""",con=conn)
#Oops, it seems that most charged apps are also rated 5.


,rating,count
0,5,330081
1,4,30593
2,3,7449
3,2,3337
4,1,10612


In [14]:
# How many apps rated 5 in total?
pd.read_sql("""

SELECT COUNT(*)
FROM (
    SELECT rating, helpful_count, price
    FROM reviews
    JOIN pricing_plans
    ON reviews.app_url = pricing_plans.app_url
    )
WHERE rating = 5

""",con=conn)
# Among all the apps rated 5, how many are free apps?
pd.read_sql("""

SELECT price,count(*) as [count]
FROM (
    SELECT rating, helpful_count, price
    FROM reviews
    JOIN pricing_plans
    ON reviews.app_url = pricing_plans.app_url
    )
WHERE (price = 'Free' OR price = 'Free to install') AND rating = 5

""",con=conn)

,COUNT(*)
0,503514


In [15]:
# Then what about the helpful_count of free apps?
pd.read_sql("""

SELECT helpful_count,count(*) as [count]
FROM (
    SELECT rating, helpful_count, price
    FROM reviews
    JOIN pricing_plans
    ON reviews.app_url = pricing_plans.app_url
    )
WHERE price = 'Free' OR price = 'Free to install'
GROUP BY helpful_count
ORDER BY helpful_count DESC

""",con=conn)

# as well as the helpful_count of charged apps
pd.read_sql("""

SELECT helpful_count,count(*) as [count]
FROM (
    SELECT rating, helpful_count, price
    FROM reviews
    JOIN pricing_plans
    ON reviews.app_url = pricing_plans.app_url
    )
WHERE price != 'Free' AND price != 'Free to install'
GROUP BY helpful_count
ORDER BY helpful_count DESC

""",con=conn)

,helpful_count,count
0,49,2
1,46,2
2,44,2
3,43,4
4,42,6
5,41,4
6,39,1
7,38,4
8,36,6
9,35,10
